In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt


import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torch.autograd import Variable

from torchvision import transforms as T
import torchvision


from PIL import Image
import cv2
import albumentations as A

import time
import os
from tqdm.notebook import tqdm

from pathlib import Path
import tifffile as tiff

import cv2 as cv
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

# import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
# utils.py

import json


def load_json( path):
    with open(path, 'r') as jsfile:
        data = json.load(jsfile)
    return data

def load_image(path, to_np=False):
    img = Image.open(path)
    if to_np:
        img = np.array(img)
    return img

def load_tiff(path):
    img = tiff.imread(path)
    return img

def class2idx():
    data = {
        "unlabeled": 0,   "paved-area": 1,   "dirt": 2,
        "grass": 3,       "gravel": 4,       "water": 5,
        "rocks": 6,       "pool": 7,         "vegetation": 8,
        "roof": 9,        "wall": 10,        "window": 11,
        "door": 12,       "fence": 13,       "fence-pole": 14,
        "person": 15,     "dog": 16,         "car": 17,
        "bicycle": 18,    "tree": 19,        "bald-tree": 20,
        "ar-marker": 21,  "obstacle": 22,    "conflicting": 23
    }
    return data

def idx2class():
    data = [
        'unlabeled','paved-area','dirt',
        'grass','gravel','water',
        'rocks','pool','vegetation',
        'roof','wall','window',
        'door','fence','fence-pole',
        'person','dog','car',
        'bicycle','tree','bald-tree',
        'ar-marker','obstacle','conflicting'
    ]
    return data

def visualize(image, mask, alpha_val=0.6, figsize_val=(10,10)):
    if type(image) == str:
        img = load_image(image)
    if type(mask) == str:
        mask = load_tiff(mask)
    
    print('Image Size', np.asarray(image).shape)
    print('Mask Size', np.asarray(mask).shape)

    plt.figure(figsize=figsize_val)
    plt.imshow(image)
    plt.imshow(mask, alpha=alpha_val)
    plt.title('Picture with Mask Appplied')
    plt.show()

In [ ]:
# dataset.py

from sklearn.model_selection import train_test_split


class DroneSegmentation(Dataset):
    def __init__(self, root, val_size=0.1, mode='train', transform=None, 
                 image_transform=None, mask_transform=None, train_size=0.2):
        self.root = Path(root)
        self.val_size = val_size
        self.train_size = train_size
        self.mode = mode
        self.transform = transform
        self.image_transform = image_transform
        self.mask_transform = mask_transform
        
        self._load_files()
        self._create_dataframe()
        self._split_dataframe()
    
    def _load_files(self):
        self.image_path = self.root.joinpath('images')
        self.mask_path = self.root.joinpath('labels/tiff')
        
        self.image_files = sorted(list(self.image_path.glob('*.jpg')))
        self.mask_files = sorted(list(self.mask_path.glob('*.tiff')))
        
    @property    
    def idx2class(self):
        return idx2class()
    
    @property
    def class2idx(self):
        return class2idx()
        
    def _create_dataframe(self):
        names, imgs_path, msks_path = [],[],[]
        for idx in range(len(self.image_files)):
            name = self.image_files[idx].stem
            img_path = str(self.image_files[idx])
            msk_path = str(self.mask_files[idx])
            names.append(name)
            imgs_path.append(img_path)
            msks_path.append(msk_path)

        data_dict = {
            'id': names,
            'image_path': imgs_path,
            'mask_path': msks_path
        }
        self.dataframe = pd.DataFrame(data_dict)
    
    def _split_dataframe(self):
        self.trainset, self.validset = train_test_split(self.dataframe, test_size=self.val_size, train_size=self.train_size, random_state=19)
        if self.mode == 'train':
            self.files = self.trainset
        else:
            self.files = self.validset
    
    def __len__(self):
        return len(self.files)
    
    def __getitem__(self, idx):
        impath = self.files['image_path'].iloc[idx]
        mspath = self.files['mask_path'].iloc[idx]
        
        orig_img = load_image(impath, to_np=True)
        mask_img = load_tiff(mspath)
        mask_img = mask_img.astype(np.uint8)
        
        if self.transform:
            orig_img, mask_img = self.transform(orig_img, mask_img)
            
        if self.image_transform:
            orig_img = self.image_transform(orig_img)
        
        if self.mask_transform:
            mask_img = self.mask_transform(mask_img)   
        
        return orig_img, mask_img



In [ ]:
# transform_pair.py

class PairCompose:
    def __init__(self, transforms):
        self.transforms = transforms

    def __call__(self, image, mask):
        for t in self.transforms:
            image, mask = t(image, mask)
        return image, mask

    def __repr__(self):
        format_string = self.__class__.__name__ + '('
        for t in self.transforms:
            format_string += '\n'
            format_string += '    {0}'.format(t)
        format_string += '\n)'
        return format_string


class PairResize:
    def __init__(self, dim, interpolation=cv.INTER_AREA):
        self.dim = dim
        self.inter = interpolation
    
    def __call__(self, image:np.ndarray, mask:np.ndarray):
        image_result = cv.resize(image, self.dim, interpolation=self.inter)
        mask_result = cv.resize(mask, self.dim, interpolation=self.inter)
        return image_result, mask_result

directory

In [ ]:
/kaggle/input/semantic-drone/semantic_drone/

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
train_tfm = PairCompose([
    PairResize(dim=(600,400)),
])

valid_tfm = PairCompose([
    PairResize(dim=(600,400)),
])

ROOT_PATH = '/kaggle/input/semantic-drone/semantic_drone/'
trainset = DroneSegmentation(root=ROOT_PATH,  mode='train', val_size=0.2, transform=train_tfm)
validset = DroneSegmentation(root=ROOT_PATH,  mode='valid', val_size=0.2, transform=valid_tfm)

In [ ]:
%%time
img, mask = validset[12]

In [ ]:
visualize(img, mask[:,:,15])
visualize(img, mask[:,:,10])